In [6]:
## jupyter notebook to simulate the range of the nuclear recoils in a given decay chain

import numpy as np
import matplotlib.pyplot as plt
import alpha_recoil_sim as ar
import importlib


In [36]:
## make dictionary containing decay chain information for a given list of isotopes
path = "/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/Sphere_Recoils_MC"

iso_list = ['ac_225', 'pb_212', 'at_211', 'th_227', 'ra_223']

iso_dict = {}

for iso in iso_list:

    iso_dict[iso] = ar.parse_decay_chain(path + "/decay_data/" + iso + "_decay_chain.txt")

In [37]:
print(iso_dict)

{'ac_225': {'Ac-225_t12': 864000.0, 'Ac-225_decays': array([[5.070e-01, 5.830e+03],
       [1.810e-01, 5.793e+03],
       [8.600e-02, 5.791e+03],
       [9.300e-02, 5.732e+03],
       [3.100e-02, 5.724e+03],
       [1.020e-01, 5.637e+03]]), 'Ac-225_daughters': ['Fr-221', 'Fr-221', 'Fr-221', 'Fr-221', 'Fr-221', 'Fr-221'], 'Fr-221_t12': 286.1, 'Fr-221_decays': array([[8.330e-01, 6.341e+03],
       [1.400e-02, 6.242e+03],
       [1.530e-01, 6.126e+03]]), 'Fr-221_daughters': ['At-217', 'At-217', 'At-217'], 'At-217_t12': 0.0323, 'At-217_decays': array([[1.000e+00, 7.067e+03]]), 'At-217_daughters': ['Bi-213'], 'Bi-213_t12': 2736.0, 'Bi-213_decays': array([[1.959e-02, 5.875e+03],
       [1.810e-03, 5.558e+03],
       [9.786e-01, 0.000e+00]]), 'Bi-213_daughters': ['Tl-209', 'Tl-209', 'Po-213'], 'Tl-209_t12': 129.60000000000002, 'Tl-209_decays': array([[1., 0.]]), 'Tl-209_daughters': ['Pb-209'], 'Po-213_t12': 3.7e-06, 'Po-213_decays': array([[1.000e+00, 8.376e+03]]), 'Po-213_daughters': ['Pb-20